# 教育集团数据宽表构建全流程
_version: 2023-01-09_

In [2]:
import pandas as pd
import numpy as np
from glob import glob

## 1 - 职业教育集团

### 1.1 示范性职业教育集团（联盟）培育单位名单

In [3]:
# 读取第一批示范性职业教育集团（联盟）培育单位名单
df_prof_1 = pd.read_excel('01-数据源/01-集团名单/教育部-第一批示范性职业教育集团（联盟）培育单位名单.xlsx', skiprows=1)

# 增加 “示范性职业教育集团（联盟）培育单位” 字段
df_prof_1['示范性职业教育集团（联盟）培育单位'] = '第一批'
df_prof_1.head()

,序号,集团名称,牵头单位,示范性职业教育集团（联盟）培育单位
0,1,联想职业教育集团,联想集团,第一批
1,2,北京交通职业教育集团,北京交通运输职业学院,第一批
2,3,北京现代服务业职业教育集团,北京祥龙资产经营有限责任公司\n北京市商业学校,第一批
3,4,北京商贸职业教育集团,北京财贸职业学院,第一批
4,5,北京现代制造业职业教育集团,北京电子科技职业学院,第一批


In [4]:
# 读取第二批示范性职业教育集团（联盟）培育单位名单
df_prof_2 = pd.read_excel('01-数据源/01-集团名单/教育部-第二批示范性职业教育集团（联盟）培育单位名单.xls', skiprows=1)

# 增加 “示范性职业教育集团（联盟）培育单位” 字段
df_prof_2['示范性职业教育集团（联盟）培育单位'] = '第二批'
df_prof_2.head()

,序号,集团名称,牵头单位,示范性职业教育集团（联盟）培育单位
0,1,中国都市农业职业教育集团,北京农业职业学院\n北京首农食品集团有限公司,第二批
1,2,北京昌平职业教育集团,北京市昌平职业学校,第二批
2,3,北京城市建设与管理职业教育集团,北京工业职业技术学院,第二批
3,4,天津交通职业教育集团,天津交通职业学院,第二批
4,5,京津冀模具现代职业教育集团,天津轻工职业技术学院,第二批


In [5]:
# 合并两批名单
df_prof = pd.concat([df_prof_1, df_prof_2], ignore_index=True)

# 增加“集团类型”字段
df_prof['集团类型'] = '职业教育'

# 多个牵头单位的情况，用逗号分隔
df_prof['牵头单位'] = df_prof['牵头单位'].str.replace('\n', '，')

# 删除“序号”列
df_prof.drop('序号', axis=1, inplace=True)
df_prof.tail()

,集团名称,牵头单位,示范性职业教育集团（联盟）培育单位,集团类型
294,甘肃省焊接专业职业教育集团,甘肃钢铁职业技术学院,第二批,职业教育
295,甘肃省智能制造职业教育集团,甘肃机电职业技术学院,第二批,职业教育
296,青海建筑通信职业教育集团,青海建筑职业技术学院,第二批,职业教育
297,宁夏现代服务业职业教育集团,宁夏财经职业技术学院,第二批,职业教育
298,新疆轨道交通职业教育集团,新疆铁道职业技术学院,第二批,职业教育


### 1.2 职业教育集团成员信息 

In [13]:
member_list = sorted(glob('01-数据源/02-成员名单/成员名单*'))
df_groups = pd.DataFrame()

for group in member_list:
    group_name = group.split('-')[-1][:-4]
    df_group = pd.read_csv(group)
    df_group['集团名称'] = group_name
    df_groups = pd.concat([df_groups, df_group], ignore_index=True)

cols = ['集团名称', '成员名称', '成员级别', '成员大类', '成员类型']
df_groups = df_groups[cols].copy()
df_groups

,集团名称,成员名称,成员级别,成员大类,成员类型
0,上海交通物流职业教育集团,上海交运日红国际物流有限公司,NaN,企业,NaN
1,上海交通物流职业教育集团,上港集团物流有限公司,NaN,企业,NaN
2,上海交通物流职业教育集团,东方航空物流有限公司,NaN,企业,NaN
3,上海交通物流职业教育集团,东方国际物流（集团）有限公司,NaN,企业,NaN
4,上海交通物流职业教育集团,锦海捷亚国际货运有限公司,NaN,企业,NaN
...,...,...,...,...,...
4390,重庆电子信息职业教育集团,泰国洛坤技术学院,理事单位,NaN,NaN
4391,重庆电子信息职业教育集团,泰国蓝彭技术学院,理事单位,NaN,NaN
4392,重庆电子信息职业教育集团,泰国北榄技术学院,理事单位,NaN,NaN
4393,重庆电子信息职业教育集团,韩国东亚大学,理事单位,NaN,NaN


In [15]:
# 统一成员大类的标签名称
label_map = {
    '企业单位': '企业', 
    '职业院校': '院校', 
    '学校': '院校',
    '政府单位': '政府部门', 
    '研究所': '科研机构',
    }
df_groups['成员大类'].replace(label_map, inplace=True)

In [16]:
df_groups['成员大类'].value_counts(dropna=False)

NaN       2266
企业        1630
院校         261
行业协会        90
企事业单位       62
学校教育机构      30
科研机构        25
政府部门        17
医院           7
研究所          2
咨询机构         2
鉴定机构         2
研究机构         1
Name: 成员大类, dtype: int64

### 1.2 职业教育集团数据补充 YS_20221214

In [5]:
# 读取补充信息表 YS_20221214
df_YS = pd.read_excel('05-补全计划/基础+职业教育集团信息补全_YS_20221214.xlsx')

# 调整部分列名
df_YS.rename(columns={'教育集团名称': '集团名称', '教育集团类型': '集团类型', 
                      '红色链接': '链接', '红色数据来源': '成员数据来源'}, inplace=True)

# 删除无效的“编号”列
df_YS = df_YS.iloc[:, 1:].copy()

# 仅保留职业教育集团数据
df_YS = df_YS[df_YS['集团类型'] == '职业教育'].copy()

# 清除文本数据前后的空格符
for col in df_YS.columns:
    if (df_YS[col].str.startswith(' ').sum() > 0) | (df_YS[col].str.endswith(' ').sum() > 0):
        print(f'“{col}” 列存在空格符, 已去除')
        df_YS[col] = df_YS[col].str.strip()

# 保存中间结果
df_YS.to_csv('03-中间结果/职业教育集团信息补全_YS_20221214_clean.csv', index=False)
df_YS.tail()

“成员名称” 列存在空格符, 已去除


,集团名称,集团类型,主管机构,牵头单位,成员名称,成员类型,黑色数据来源,成员数据来源,链接,其他备注
6025,福建林业职教集团,职业教育,福建省,福建林业职业技术学院,NaN,NaN,当地教育厅或政府公告,NaN,NaN,没有官网
6026,福建省信息职业教育集团,职业教育,福建省,福建信息职业技术学院,NaN,NaN,当地教育厅或政府公告,NaN,NaN,没有官网
6027,福建省闽台汽车工业职教集团,职业教育,福建省,福建理工学校,NaN,NaN,当地教育厅或政府公告,NaN,NaN,没有官网
6028,厦门市电子信息职业教育集团,职业教育,福建省,厦门信息学校,NaN,NaN,当地教育厅或政府公告,NaN,NaN,没有官网
6029,福建交通物流职业教育集团,职业教育,福建省,福建船政交通职业学院,NaN,NaN,当地教育厅或政府公告,NaN,NaN,没有官网


In [6]:
# 提取并保存集团信息
keep_cols = ['集团名称', '主管机构', '成员数据来源', '链接']
df_group = df_YS.drop_duplicates('集团名称', keep='first')[keep_cols].reset_index(drop=True)
df_group.to_csv('03-中间结果/职业教育集团信息补全_YS_20221214_集团信息.csv', index=False)
df_group.head()

,集团名称,主管机构,成员数据来源,链接
0,联想职业教育集团,北京市,北京市教育委员会,http://jw.beijing.gov.cn/jyzx/jyxw/201606/t201...
1,北京交通职业教育集团,北京市,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...
2,北京现代服务业职业教育集团,北京市,北京现代服务业职业教育集团,https://zjjt.bjsx.com.cn/jtjj/yxcy.htm
3,北京商贸职业教育集团,北京市,北京财贸职业学院,https://www.bjczy.edu.cn/folder276/folder546/2...
4,北京现代制造业职业教育集团,北京市,NaN,NaN


In [7]:
# 删除没找到成员信息的数据
df_YS.dropna(subset=['成员名称'], axis=0, inplace=True)

# 提取成员信息
keep_cols = ['集团名称', '成员名称', '成员类型']
df_member = df_YS[keep_cols].copy()

# 成员名称去重
df_member.drop_duplicates('成员名称', keep='first', inplace=True)

# 保存成员信息
df_member.to_csv('03-中间结果/职业教育集团信息补全_YS_20221214_成员信息.csv', index=False)
df_member.head()

,集团名称,成员名称,成员类型
0,联想职业教育集团,北京市昌平职业学校,中职
1,北京交通职业教育集团,北京铁路电气化学校,中职
2,北京交通职业教育集团,北京地铁技术学校,中职
3,北京交通职业教育集团,北京市公共交通高级技工学校,中职
4,北京交通职业教育集团,北京电气工程学校,中职


### 1.3 职业教育集团数据补充 LJJ_20221215

In [8]:
# 读取补充信息表 LJJ_20221215
df_LJJ = pd.read_excel('05-补全计划/职业教育集团信息补全_LJJ_20221215.xlsx')

# 调整部分列名
df_LJJ.rename(columns={'教育集团名称': '集团名称', '成员类别': '成员类型', '数据来源': '成员数据来源'}, inplace=True)

# 删除无效的前两列
df_LJJ = df_LJJ.iloc[:, 2:].copy()

# 删除没找到成员信息的数据
df_LJJ.dropna(subset=['成员名称'], axis=0, inplace=True)

# 清除文本数据前后的空格符
for col in df_LJJ.columns:
    if (df_LJJ[col].str.startswith(' ').sum() > 0) | (df_LJJ[col].str.endswith(' ').sum() > 0):
        print(f'“{col}” 列存在空格符, 已去除')
        df_LJJ[col] = df_LJJ[col].str.strip()
        
# 保存中间结果
df_LJJ.to_csv('03-中间结果/职业教育集团信息补全_LJJ_20221215_clean.csv', index=False)
df_LJJ.tail()

“成员名称” 列存在空格符, 已去除


,集团名称,教育集团类型,主管机构,牵头单位,成员名称,成员类型,成员数据来源,链接,备注
1540,NaN,NaN,NaN,NaN,德阳市第二人民医院,NaN,官网,http://zjlm.sxyyc.net/sanxia-portal/view/index...,NaN
1541,NaN,NaN,NaN,NaN,重庆三峡医专附属中医院,NaN,官网,http://zjlm.sxyyc.net/sanxia-portal/view/index...,NaN
1542,NaN,NaN,NaN,NaN,重庆忆念美美容管理有限公司,NaN,官网,http://zjlm.sxyyc.net/sanxia-portal/view/index...,NaN
1543,NaN,NaN,NaN,NaN,忠县同怡医院,NaN,官网,http://zjlm.sxyyc.net/sanxia-portal/view/index...,NaN
1544,NaN,NaN,NaN,NaN,重庆万和连锁药房有限公司,NaN,官网,http://zjlm.sxyyc.net/sanxia-portal/view/index...,NaN


In [9]:
# 提取并保存集团信息
keep_cols = ['集团名称', '主管机构', '成员数据来源', '链接']
df_group = df_LJJ[df_LJJ['集团名称'].notnull()][keep_cols].copy().reset_index(drop=True)
df_group.to_csv('03-中间结果/职业教育集团信息补全_LJJ_20221215_集团信息.csv', index=False)
df_group.head()

,集团名称,主管机构,成员数据来源,链接
0,全国现代服务业职业教育集团,NaN,官网,http://cmsiveg.org/index.aspx
1,甘肃省汽车行业职教集团,甘肃省,官网,http://www.lzpcc.edu.cn/qczjjt/jtgk/cydw.htm
2,咸阳职业教育集团,陕西省，咸阳市,官网,https://zjjt.xianyangzhiyuan.edu.cn/index.htm
3,汉中职业教育集团,陕西省,NaN,http://jxcg.hzvtc.cn/info/1018/1117.htm
4,云南省曲靖市麒麟职业教育集团,云南省,NaN,http://www.qjou.cn/channels/385.html


In [10]:
# 提取成员信息
keep_cols = ['集团名称', '成员名称', '成员类型']
df_member = df_LJJ[df_LJJ['成员名称'].notnull()][keep_cols].copy().reset_index(drop=True)

# 成员名称去重
df_member.drop_duplicates('成员名称', keep='first', inplace=True)

# 修正部分成员名称的错误
df_member['成员名称'] = df_member['成员名称'].str.replace('简介', '')

# 填充省略掉的“教育集团名称”
df_member['集团名称'].fillna(method='ffill', inplace=True)

# 保存成员信息
df_member.to_csv('03-中间结果/职业教育集团信息补全_LJJ_20221215_成员信息.csv', index=False)
df_member.head()

,集团名称,成员名称,成员类型
0,全国现代服务业职业教育集团,PMI中国,社会团体
1,全国现代服务业职业教育集团,安徽城市管理职业学院,院校
2,全国现代服务业职业教育集团,安徽工商职业学院,院校
3,全国现代服务业职业教育集团,安徽国际商务职业教育集团,职教集团
4,全国现代服务业职业教育集团,安徽国际商务职业学院,院校


In [11]:
# 合并现有全部集团信息
df_group_1 = pd.read_csv('03-中间结果/职业教育集团信息补全_YS_20221214_集团信息.csv')
df_group_2 = pd.read_csv('03-中间结果/职业教育集团信息补全_LJJ_20221215_集团信息.csv')
df_group = pd.concat([df_group_1, df_group_2], ignore_index=True)

# 去除重复信息
df_group.drop_duplicates('集团名称', keep='last', inplace=True)
df_group.tail()

,集团名称,主管机构,成员数据来源,链接
329,中山市专业镇特色产业职业教育集团,广东省，中山市,官网,https://www.zspt.edu.cn/zszjjt/jtjj/jtcy.htm
330,湖南现代畜牧养殖职业教育集团,湖南省,官网,http://www.hnxmedu.com/catid/12_1.html
331,重庆现代服务业职业教育集团,重庆市,官网,http://www.cqxdfwyzjjt.com/main.psp
332,广东纺织职业教育集团,广东省,官网,https://www.gdpt.edu.cn/al_40/1289
333,重庆三峡库区医教协同职业教育集团,重庆市,官网,http://zjlm.sxyyc.net/sanxia-portal/view/index...


In [12]:
# 合并现有全部成员信息
df_member_1 = pd.read_csv('03-中间结果/职业教育集团信息补全_YS_20221214_成员信息.csv')
df_member_2 = pd.read_csv('03-中间结果/职业教育集团信息补全_LJJ_20221215_成员信息.csv')
df_member = pd.concat([df_member_1, df_member_2], ignore_index=True)

# 去除重复信息
df_member.drop_duplicates(subset=['集团名称','成员名称'], keep='last', inplace=True)
df_member = df_member.reset_index(drop=True)
df_member.tail()

,集团名称,成员名称,成员类型
6327,重庆三峡库区医教协同职业教育集团,德阳市第二人民医院,NaN
6328,重庆三峡库区医教协同职业教育集团,重庆三峡医专附属中医院,NaN
6329,重庆三峡库区医教协同职业教育集团,重庆忆念美美容管理有限公司,NaN
6330,重庆三峡库区医教协同职业教育集团,忠县同怡医院,NaN
6331,重庆三峡库区医教协同职业教育集团,重庆万和连锁药房有限公司,NaN


### 1.3 数据合并

In [13]:
# 将集团信息合并至现有职业集团名单
df_prof = df_prof.merge(df_group, how='left', on='集团名称')
df_prof.head()

,集团名称,牵头单位,示范性职业教育集团（联盟）培育单位,集团类型,主管机构,成员数据来源,链接
0,联想职业教育集团,联想集团,第一批,职业教育,北京市,北京市教育委员会,http://jw.beijing.gov.cn/jyzx/jyxw/201606/t201...
1,北京交通职业教育集团,北京交通运输职业学院,第一批,职业教育,北京市,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...
2,北京现代服务业职业教育集团,北京祥龙资产经营有限责任公司，北京市商业学校,第一批,职业教育,北京市,北京现代服务业职业教育集团,https://zjjt.bjsx.com.cn/jtjj/yxcy.htm
3,北京商贸职业教育集团,北京财贸职业学院,第一批,职业教育,北京市,北京财贸职业学院,https://www.bjczy.edu.cn/folder276/folder546/2...
4,北京现代制造业职业教育集团,北京电子科技职业学院,第一批,职业教育,北京市,NaN,NaN


In [14]:
# 将成员信息合并至现有职业集团名单
df_prof = df_prof.merge(df_member, how='left', on='集团名称')
df_prof.tail()

,集团名称,牵头单位,示范性职业教育集团（联盟）培育单位,集团类型,主管机构,成员数据来源,链接,成员名称,成员类型
5952,甘肃省焊接专业职业教育集团,甘肃钢铁职业技术学院,第二批,职业教育,甘肃省,NaN,NaN,NaN,NaN
5953,甘肃省智能制造职业教育集团,甘肃机电职业技术学院,第二批,职业教育,甘肃省,NaN,NaN,NaN,NaN
5954,青海建筑通信职业教育集团,青海建筑职业技术学院,第二批,职业教育,青海省,NaN,NaN,NaN,NaN
5955,宁夏现代服务业职业教育集团,宁夏财经职业技术学院,第二批,职业教育,宁夏,NaN,NaN,NaN,NaN
5956,新疆轨道交通职业教育集团,新疆铁道职业技术学院,第二批,职业教育,新疆,NaN,NaN,NaN,NaN


In [15]:
# 保存宽表数据
df_prof.to_csv('03-中间结果/教育集团数据宽表-职业教育.csv', index=False)
df_prof[df_prof['成员名称'].notna()]

,集团名称,牵头单位,示范性职业教育集团（联盟）培育单位,集团类型,主管机构,成员数据来源,链接,成员名称,成员类型
0,联想职业教育集团,联想集团,第一批,职业教育,北京市,北京市教育委员会,http://jw.beijing.gov.cn/jyzx/jyxw/201606/t201...,北京市昌平职业学校,中职
1,北京交通职业教育集团,北京交通运输职业学院,第一批,职业教育,北京市,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...,北京铁路电气化学校,中职
2,北京交通职业教育集团,北京交通运输职业学院,第一批,职业教育,北京市,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...,北京地铁技术学校,中职
3,北京交通职业教育集团,北京交通运输职业学院,第一批,职业教育,北京市,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...,北京市公共交通高级技工学校,中职
4,北京交通职业教育集团,北京交通运输职业学院,第一批,职业教育,北京市,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...,北京电气工程学校,中职
...,...,...,...,...,...,...,...,...,...
5946,甘肃省汽车行业职教集团,兰州石化职业技术学院,第二批,职业教育,甘肃省,官网,http://www.lzpcc.edu.cn/qczjjt/jtgk/cydw.htm,陕西省西咸新区沣东新城管委会,政府部门
5947,甘肃省汽车行业职教集团,兰州石化职业技术学院,第二批,职业教育,甘肃省,官网,http://www.lzpcc.edu.cn/qczjjt/jtgk/cydw.htm,陕西省西咸新区沣西新城管委会,政府部门
5948,甘肃省汽车行业职教集团,兰州石化职业技术学院,第二批,职业教育,甘肃省,官网,http://www.lzpcc.edu.cn/qczjjt/jtgk/cydw.htm,陕西省西咸新区空港新城管委会,政府部门
5949,甘肃省汽车行业职教集团,兰州石化职业技术学院,第二批,职业教育,甘肃省,官网,http://www.lzpcc.edu.cn/qczjjt/jtgk/cydw.htm,陕西省西咸新区秦汉新城管委会,政府部门


## 2 - 基础教育集团

### 2.1 基础教育集团数据补充 YS_20221214

In [16]:
# 读取补充信息表
df_YS = pd.read_excel('05-补全计划/基础+职业教育集团信息补全_YS_20221214.xlsx')

# 调整部分列名
df_YS.rename(columns={'教育集团名称': '集团名称', '教育集团类型': '集团类型', 
                      '红色链接': '链接', '红色数据来源': '成员数据来源'}, inplace=True)

# 删除无效的“编号”列
df_YS = df_YS.iloc[:, 1:].copy()

# 仅保留基础教育集团数据
df_YS = df_YS[df_YS['集团类型'] == '基础教育'].copy()

# 清除文本数据前后的空格符
for col in df_YS.columns:
    if (df_YS[col].str.startswith(' ').sum() > 0) | (df_YS[col].str.endswith(' ').sum() > 0):
        print(f'“{col}” 列存在空格符, 已去除')
        df_YS[col] = df_YS[col].str.strip()

# 保存中间结果
df_YS.to_csv('03-中间结果/基础教育集团信息补全_YS_20221214_clean.csv', index=False)
df_YS.tail()

“成员名称” 列存在空格符, 已去除


,集团名称,集团类型,主管机构,牵头单位,成员名称,成员类型,黑色数据来源,成员数据来源,链接,其他备注
6486,长垣市第一初级中学教育集团,基础教育,河南省，长垣市,NaN,NaN,NaN,当地教育厅或政府公告,NaN,NaN,NaN
6487,邓州市花洲实验小学教育集团,基础教育,河南省，邓州市,NaN,NaN,NaN,当地教育厅或政府公告,NaN,NaN,NaN
6488,河南省第二实验中学教育集团,基础教育,河南省，厅直属实验学校,NaN,NaN,NaN,当地教育厅或政府公告,NaN,NaN,NaN
6489,河南省实验小学教育集团,基础教育,河南省，厅直属实验学校,NaN,NaN,NaN,当地教育厅或政府公告,NaN,NaN,NaN
6490,河南师范大学附属中学教育集团,基础教育,河南省，厅直属实验学校,NaN,NaN,NaN,当地教育厅或政府公告,NaN,NaN,NaN


In [17]:
# 保存有效数据
keep_cols = ['集团名称', '集团类型', '主管机构', '牵头单位', '成员名称', '成员类型', '成员数据来源', '链接']
df_base = df_YS[keep_cols].copy().reset_index(drop=True)
df_base.tail()

,集团名称,集团类型,主管机构,牵头单位,成员名称,成员类型,成员数据来源,链接
456,长垣市第一初级中学教育集团,基础教育,河南省，长垣市,NaN,NaN,NaN,NaN,NaN
457,邓州市花洲实验小学教育集团,基础教育,河南省，邓州市,NaN,NaN,NaN,NaN,NaN
458,河南省第二实验中学教育集团,基础教育,河南省，厅直属实验学校,NaN,NaN,NaN,NaN,NaN
459,河南省实验小学教育集团,基础教育,河南省，厅直属实验学校,NaN,NaN,NaN,NaN,NaN
460,河南师范大学附属中学教育集团,基础教育,河南省，厅直属实验学校,NaN,NaN,NaN,NaN,NaN


### 2.2 基础教育集团数据补充 LJJ_20221216

In [18]:
# 导入人工补全后的数据
df_fill = pd.read_excel('05-补全计划/基础教育集团信息补全_LJJ_20221216.xlsx', sheet_name='Sheet1')
df_fill.head()

,集团名称,牵头单位,集团类型,主管机构,成员数据来源,链接,成员名称,成员类型,备注
0,丹东市实验小学教育集团,丹东市实验小学,基础教育,辽宁省，丹东市,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...,丹东市实验小学新区分校,小学,NaN
1,NaN,NaN,NaN,NaN,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...,育鹏小学,小学,NaN
2,NaN,NaN,NaN,NaN,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...,九中附小,小学,NaN
3,NaN,NaN,NaN,NaN,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...,宽甸县虎山镇中心小学,小学,结对校
4,广州执信中学教育集团,NaN,基础教育,广东省，广州市,百家号,https://baijiahao.baidu.com/s?id=1739677974237...,执信中学从化实验学校,NaN,NaN


In [19]:
# 提取并保存集团信息
keep_cols = ['集团名称', '牵头单位', '主管机构', '成员数据来源', '链接']
df_group = df_fill[df_fill['集团名称'].notnull()][keep_cols].copy().reset_index(drop=True)
df_group.to_csv('03-中间结果/基础教育集团信息补全_LJJ_20221216_集团信息.csv', index=False)
df_group.head()

,集团名称,牵头单位,主管机构,成员数据来源,链接
0,丹东市实验小学教育集团,丹东市实验小学,辽宁省，丹东市,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...
1,广州执信中学教育集团,NaN,广东省，广州市,百家号,https://baijiahao.baidu.com/s?id=1739677974237...
2,广东实验中学教育集团,NaN,广东省,百家号,https://baijiahao.baidu.com/s?id=1739677974237...
3,广铁一中教育集团,广州市铁一中学,广东省，广州市,百家号,https://baijiahao.baidu.com/s?id=1739677974237...
4,广东广雅教育集团,NaN,广东省，广州市,百家号,https://baijiahao.baidu.com/s?id=1739677974237...


In [20]:
# 提取成员信息
keep_cols = ['集团名称', '成员名称', '成员类型']
df_member = df_fill[df_fill['成员名称'].notnull()][keep_cols].copy().reset_index(drop=True)

# 填充省略掉的“教育集团名称”
df_member['集团名称'].fillna(method='ffill', inplace=True)

# 保存成员信息
df_member.to_csv('03-中间结果/基础教育集团信息补全_LJJ_20221216_成员信息.csv', index=False)
df_member.head()

,集团名称,成员名称,成员类型
0,丹东市实验小学教育集团,丹东市实验小学新区分校,小学
1,丹东市实验小学教育集团,育鹏小学,小学
2,丹东市实验小学教育集团,九中附小,小学
3,丹东市实验小学教育集团,宽甸县虎山镇中心小学,小学
4,广州执信中学教育集团,执信中学从化实验学校,NaN


In [21]:
# 将集团信息与成员信息合并
df_base_new = df_group.merge(df_member, on='集团名称', how='left')
df_base_new['集团类型'] = '基础教育'
df_base_new.head()

,集团名称,牵头单位,主管机构,成员数据来源,链接,成员名称,成员类型,集团类型
0,丹东市实验小学教育集团,丹东市实验小学,辽宁省，丹东市,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...,丹东市实验小学新区分校,小学,基础教育
1,丹东市实验小学教育集团,丹东市实验小学,辽宁省，丹东市,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...,育鹏小学,小学,基础教育
2,丹东市实验小学教育集团,丹东市实验小学,辽宁省，丹东市,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...,九中附小,小学,基础教育
3,丹东市实验小学教育集团,丹东市实验小学,辽宁省，丹东市,教育局官网,https://ddedu.dandong.gov.cn/html/DDJYJ/202104...,宽甸县虎山镇中心小学,小学,基础教育
4,广州执信中学教育集团,NaN,广东省，广州市,百家号,https://baijiahao.baidu.com/s?id=1739677974237...,执信中学从化实验学校,NaN,基础教育


### 2.3 数据合并

In [22]:
# 从现有基础教育集团名单中去掉补充的集团
df_base_old = df_base[~df_base['集团名称'].isin(df_base_new['集团名称'])]

# 将补充数据与现有名单合并
df_base = pd.concat([df_base_old, df_base_new], ignore_index=True)
df_base.tail()

,集团名称,集团类型,主管机构,牵头单位,成员名称,成员类型,成员数据来源,链接
2783,天津市第五中学教育集团,基础教育,天津市,NaN,天津市红桥小学,NaN,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...
2784,天津市第五中学教育集团,基础教育,天津市,NaN,红桥区洪湖里小学,NaN,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...
2785,天津市民族中学教育集团,基础教育,天津市,NaN,天津市民族中学,NaN,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...
2786,天津市民族中学教育集团,基础教育,天津市,NaN,红桥区跃进里小学,NaN,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...
2787,天津市民族中学教育集团,基础教育,天津市,NaN,红桥区邵公庄小学,NaN,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...


In [23]:
# 保存基础教育集团宽表
df_base.to_csv('03-中间结果/教育集团数据宽表-基础教育.csv', index=False)

## 3 数据补全与修正

### 3.1 职业教育集团-成员类型批量补全和修正

In [24]:
# 使用高等教育宽表中的信息补充部分缺失的成员类型
df_high = pd.read_csv('../数据收集-高等教育/04-数据宽表/高等教育数据宽表v20230103.csv')
df_high = df_high[['学校名称', '办学层次']]
df_high['办学层次'].replace('专科', '专科（高职）院校', inplace=True)
df_prof = df_prof.merge(df_high, left_on='成员名称', right_on='学校名称', how='left')

In [25]:
# 优先使用办学层次字段补全成员类型
has_level = df_prof['办学层次'].notna()
no_member_type = df_prof['成员类型'].isna()
has_member = df_prof['成员名称'].notna()
mask = has_level & has_member & no_member_type
df_prof.loc[mask, '成员类型'] = df_prof.loc[mask, '办学层次']

In [26]:
# 修正被分类为企业的医院
name_hospital = df_prof['成员名称'].str.endswith('医院')
label_company = df_prof['成员类型'] == '企业'
mask = name_hospital & label_company
df_prof.loc[mask, '成员类型'] = '事业单位'

# 修正被分类为企业的幼儿园
name_kid = df_prof['成员名称'].str.endswith('幼儿园')
label_company = df_prof['成员类型'] == '企业'
mask = name_kid & label_company
df_prof.loc[mask, '成员类型'] = '幼儿园'

In [35]:
# 统一成员类型标签
label_dict = {
    '高职': '专科（高职）院校', 
    '高职专科院校': '专科（高职）院校',
    '中职': '中职院校',
    '中职学校': '中职院校',
    '本科': '本科院校',
    '政府': '政府部门',
    '企业': '企业单位',
}
df_prof['成员类型'].replace(label_dict, inplace=True)

In [36]:
df_prof['成员类型'].value_counts()

企业单位          2926
专科（高职）院校       387
行业协会           179
中职院校           175
政府部门            90
本科院校            79
研究机构            70
企业机构            67
学校教育机构          30
事业单位            16
行业协会（政府单位）      15
职业院校            15
科研院所、院校         14
其他组织            14
行业协会或学会          9
科研院所             3
幼儿园              2
Name: 成员类型, dtype: int64

In [34]:
# 更新职业教育集团宽表数据
df_prof.to_csv('03-中间结果/教育集团数据宽表-职业教育.csv', index=False)
df_prof[df_prof['成员名称'].notna()]

,集团名称,牵头单位,示范性职业教育集团（联盟）培育单位,集团类型,主管机构,成员数据来源,链接,成员名称,成员类型,学校名称,办学层次
0,联想职业教育集团,联想集团,第一批,职业教育,北京市,北京市教育委员会,http://jw.beijing.gov.cn/jyzx/jyxw/201606/t201...,北京市昌平职业学校,中职院校,NaN,NaN
1,北京交通职业教育集团,北京交通运输职业学院,第一批,职业教育,北京市,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...,北京铁路电气化学校,中职院校,NaN,NaN
2,北京交通职业教育集团,北京交通运输职业学院,第一批,职业教育,北京市,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...,北京地铁技术学校,中职院校,NaN,NaN
3,北京交通职业教育集团,北京交通运输职业学院,第一批,职业教育,北京市,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...,北京市公共交通高级技工学校,中职院校,NaN,NaN
4,北京交通职业教育集团,北京交通运输职业学院,第一批,职业教育,北京市,北京交通运输职业学院,https://www.bjjt.edu.cn/jthbx1/bjjtzyjyjt/cydw...,北京电气工程学校,中职院校,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5946,甘肃省汽车行业职教集团,兰州石化职业技术学院,第二批,职业教育,甘肃省,官网,http://www.lzpcc.edu.cn/qczjjt/jtgk/cydw.htm,陕西省西咸新区沣东新城管委会,政府部门,NaN,NaN
5947,甘肃省汽车行业职教集团,兰州石化职业技术学院,第二批,职业教育,甘肃省,官网,http://www.lzpcc.edu.cn/qczjjt/jtgk/cydw.htm,陕西省西咸新区沣西新城管委会,政府部门,NaN,NaN
5948,甘肃省汽车行业职教集团,兰州石化职业技术学院,第二批,职业教育,甘肃省,官网,http://www.lzpcc.edu.cn/qczjjt/jtgk/cydw.htm,陕西省西咸新区空港新城管委会,政府部门,NaN,NaN
5949,甘肃省汽车行业职教集团,兰州石化职业技术学院,第二批,职业教育,甘肃省,官网,http://www.lzpcc.edu.cn/qczjjt/jtgk/cydw.htm,陕西省西咸新区秦汉新城管委会,政府部门,NaN,NaN


In [42]:
# 筛选出职业教育集团成员类型缺失的数据
is_prof = df_wide['集团类型'] == '职业教育'
no_member_type = df_wide['成员类型'].isna()
has_member = df_wide['成员名称'].notna()
mask = is_prof & no_member_type & has_member
df_wide[mask]['成员类型'].value_counts()

Series([], Name: 成员类型, dtype: int64)

### 3.1 合并职业教育集团与基础教育集团数据

In [32]:
# 合并职业教育集团与基础教育集团数据
df_wide = pd.concat([df_prof, df_base], ignore_index=True)
df_wide.tail()

,集团名称,牵头单位,示范性职业教育集团（联盟）培育单位,集团类型,主管机构,成员数据来源,链接,成员名称,成员类型
8740,天津市第五中学教育集团,NaN,NaN,基础教育,天津市,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...,天津市红桥小学,NaN
8741,天津市第五中学教育集团,NaN,NaN,基础教育,天津市,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...,红桥区洪湖里小学,NaN
8742,天津市民族中学教育集团,NaN,NaN,基础教育,天津市,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...,天津市民族中学,NaN
8743,天津市民族中学教育集团,NaN,NaN,基础教育,天津市,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...,红桥区跃进里小学,NaN
8744,天津市民族中学教育集团,NaN,NaN,基础教育,天津市,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...,红桥区邵公庄小学,NaN


### 3.2 数据和字段调整

In [ ]:
# 将主管机构中的省市信息独立拆分
df_wide['主管机构_省级'] = df_wide['主管机构'].apply(lambda x: x.split('，')[0])
city_idx = df_wide['主管机构'].str.contains('，')
df_wide.loc[city_idx, '主管机构_市级'] = df_wide.loc[city_idx, '主管机构'].apply(lambda x: x.split('，')[1])
df_wide.tail()

### 3.4 宽表结果保存

In [26]:
df_wide.to_csv('04-数据宽表/教育集团数据宽表v20230105.csv', index=False)
df_wide.tail()

,集团名称,牵头单位,示范性职业教育集团（联盟）培育单位,集团类型,主管机构,成员数据来源,链接,成员名称,成员类型,主管机构_省级,主管机构_市级
8740,天津市第五中学教育集团,NaN,NaN,基础教育,天津市,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...,天津市红桥小学,NaN,天津市,NaN
8741,天津市第五中学教育集团,NaN,NaN,基础教育,天津市,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...,红桥区洪湖里小学,NaN,天津市,NaN
8742,天津市民族中学教育集团,NaN,NaN,基础教育,天津市,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...,天津市民族中学,NaN,天津市,NaN
8743,天津市民族中学教育集团,NaN,NaN,基础教育,天津市,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...,红桥区跃进里小学,NaN,天津市,NaN
8744,天津市民族中学教育集团,NaN,NaN,基础教育,天津市,百家号：凤凰网房产,https://baijiahao.baidu.com/s?id=1718002390895...,红桥区邵公庄小学,NaN,天津市,NaN


In [ ]:
# 将主管机构中的省市信息独立拆分
df_wide['主管机构_省级'] = df_wide['主管机构'].apply(lambda x: x.split('，')[0])
city_idx = df_wide['主管机构'].str.contains('，')
df_wide.loc[city_idx, '主管机构_市级'] = df_wide.loc[city_idx, '主管机构'].apply(lambda x: x.split('，')[1])
df_wide.tail()

## 4 补全计划

### 4.1 职业教育集团成员类型补全_20230105

In [27]:
# 筛选出职业教育集团成员类型缺失的数据
is_prof = df_wide['集团类型'] == '职业教育'
no_member_type = df_wide['成员类型'].isna()
has_member = df_wide['成员名称'].notna()
df_prof_miss = df_wide[is_prof & no_member_type & has_member]

# 导出待补全的空表
use_cols = ['集团名称', '成员数据来源', '链接', '成员名称']
df_prof_miss[use_cols].to_csv('05-补全计划/职业教育集团成员类型补全_20230105.csv', index=False)
df_prof_miss[use_cols].tail()

,集团名称,成员数据来源,链接,成员名称
5889,甘肃省汽车行业职教集团,官网,http://www.lzpcc.edu.cn/qczjjt/jtgk/cydw.htm,宁波奥博汽车电器有限公司
5890,甘肃省汽车行业职教集团,官网,http://www.lzpcc.edu.cn/qczjjt/jtgk/cydw.htm,甘肃省畜牧职业技术学院
5891,甘肃省汽车行业职教集团,官网,http://www.lzpcc.edu.cn/qczjjt/jtgk/cydw.htm,甘肃有色冶金职业技术学院
5892,甘肃省汽车行业职教集团,官网,http://www.lzpcc.edu.cn/qczjjt/jtgk/cydw.htm,甘肃工业职业技术学院
5893,甘肃省汽车行业职教集团,官网,http://www.lzpcc.edu.cn/qczjjt/jtgk/cydw.htm,甘南州中等职业学校


### 4.2 基础教育集团 TBD_20221220

In [28]:
# 提取基础教育集团中的缺失数据
base_miss = (df_wide['集团类型']=='基础教育') & (df_wide['成员名称'].isna())
df_base_miss = df_wide[base_miss].drop('示范性职业教育集团（联盟）培育单位', axis=1)
df_base_miss.to_csv('05-补全计划/基础教育集团信息补全_TBD_20221220_空表.csv', index=False)
df_base_miss

,集团名称,牵头单位,集团类型,主管机构,成员数据来源,链接,成员名称,成员类型,主管机构_省级,主管机构_市级
6093,丹东三幼教育集团,NaN,基础教育,辽宁省，丹东市,NaN,NaN,NaN,幼儿园,辽宁省,丹东市
6094,丹东五中教育集团,NaN,基础教育,辽宁省，丹东市,NaN,NaN,NaN,初中,辽宁省,丹东市
6095,丹东六中教育集团,NaN,基础教育,辽宁省，丹东市,NaN,NaN,NaN,初中,辽宁省,丹东市
6096,丹东十三中教育集团,NaN,基础教育,辽宁省，丹东市,NaN,NaN,NaN,初中,辽宁省,丹东市
6097,丹东十七中教育集团,NaN,基础教育,辽宁省，丹东市,NaN,NaN,NaN,初中,辽宁省,丹东市
...,...,...,...,...,...,...,...,...,...,...
6386,长垣市第一初级中学教育集团,NaN,基础教育,河南省，长垣市,NaN,NaN,NaN,NaN,河南省,长垣市
6387,邓州市花洲实验小学教育集团,NaN,基础教育,河南省，邓州市,NaN,NaN,NaN,NaN,河南省,邓州市
6388,河南省第二实验中学教育集团,NaN,基础教育,河南省，厅直属实验学校,NaN,NaN,NaN,NaN,河南省,厅直属实验学校
6389,河南省实验小学教育集团,NaN,基础教育,河南省，厅直属实验学校,NaN,NaN,NaN,NaN,河南省,厅直属实验学校
